In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def gaussian_blur(image_tensor: torch.Tensor, kernel_size=5, sigma=1.0):
    channels, height, width = image_tensor.size()

    # Create 2D Gaussian kernel
    grid = torch.linspace(-kernel_size // 2 + 1, kernel_size // 2, kernel_size)
    gaussian_kernel = torch.exp(-((grid[:, None] ** 2 + grid[None, :] ** 2) / (2 * sigma ** 2)))
    gaussian_kernel /= gaussian_kernel.sum()

    # Create a 4D weight tensor for convolution
    gaussian_kernel = gaussian_kernel.expand(channels, 1, kernel_size, kernel_size)
    weight = gaussian_kernel.to(device)

    # Apply Gaussian blur
    image_tensor = image_tensor.unsqueeze(0)
    blurred_image = F.conv2d(image_tensor, weight, padding=kernel_size // 2, groups=channels)
    return blurred_image.squeeze(0)

def gaussian_noise(image_tensor: torch.Tensor, mean=0.0, std=0.1):
    noise = torch.randn(image_tensor.size(), device=device) * std + mean
    noisy_image = image_tensor + noise
    
    return torch.clamp(noisy_image, 0, 1)

In [ ]:
toTensor = transforms.ToTensor()
toImage = transforms.ToPILImage()
toGrayscale = transforms.Grayscale()

In [ ]:
image = Image.open("img1.png")

image_tensor = toGrayscale(toTensor(image)[0:3]).to(device)
blurred_image_tensor = gaussian_blur(image_tensor)
noisy_image_tensor = gaussian_noise(image_tensor)

image = toImage(image_tensor)
blurred_image = toImage(blurred_image_tensor)
noisy_image = toImage(noisy_image_tensor)

In [ ]:
_, height, width = image_tensor.shape

line = image_tensor[0][height // 2]

In [ ]:
image

In [ ]:
plt.figure(figsize=(12, 5))

plt.plot(range(len(line)), line.numpy());

In [ ]:
# Intensity profile

# Add noise to image and use smoothing to get intensity profile